In [3]:
import os
os.chdir("/content/drive/MyDrive/Colab/Callback")

In [4]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [5]:
mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

4422102/4422102 [==============================] - 0s 0us/step


In [6]:
os.makedirs("training_3", exist_ok=True)
ckpt_callback = ModelCheckpoint(filepath='training_3/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss')

In [7]:
def scheduler(epoch):
    if epoch < 2:
        return 0.001
    elif epoch < 6:
        return 0.0001
    else:
        return 0.00001

learning_rate_scheduler = LearningRateScheduler(scheduler, verbose=1)

In [8]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=3, validation_split=0.2,
                    callbacks=[learning_rate_scheduler, ckpt_callback])


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/3
1500/1500 [==============================] - 10s 3ms/step - loss: 0.4901 - accuracy: 0.8271 - val_loss: 0.4018 - val_accuracy: 0.8518 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/3
  29/1500 [..............................] - ETA: 5s - loss: 0.3750 - accuracy: 0.8631

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1500/1500 [==============================] - 6s 4ms/step - loss: 0.3661 - accuracy: 0.8677 - val_loss: 0.3682 - val_accuracy: 0.8642 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0001.
Epoch 3/3
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2887 - accuracy: 0.8947 - val_loss: 0.3197 - val_accuracy: 0.8833 - lr: 1.0000e-04


# Resuming a Training Process

In [9]:
checkpoint_path = 'training_3/weights.03-0.32.hdf5'

In [10]:
del model  # deletes the existing model

In [11]:
# returns a compiled model
# identical to the previous one
model = load_model(checkpoint_path)

In [12]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=1, validation_split=0.2,
                    callbacks=[learning_rate_scheduler])


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
1500/1500 [==============================] - 7s 4ms/step - loss: 0.3302 - accuracy: 0.8783 - val_loss: 0.3467 - val_accuracy: 0.8741 - lr: 0.0010


Oops! Look at the learning rates!

In [13]:
del model  # deletes the existing model

In [14]:
# returns a compiled model
# identical to the previous one
model = load_model(checkpoint_path)

In [15]:
def get_init_epoch(path):
    return int(path.split('-')[-2].split('.')[-1])

In [16]:
# Finding the epoch index from which we are resuming
initial_epoch = get_init_epoch(checkpoint_path)

In [17]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=4, validation_split=0.2,
                    callbacks=[learning_rate_scheduler], initial_epoch=initial_epoch)


Epoch 4: LearningRateScheduler setting learning rate to 0.0001.
Epoch 4/4
1500/1500 [==============================] - 6s 3ms/step - loss: 0.2771 - accuracy: 0.8982 - val_loss: 0.3152 - val_accuracy: 0.8868 - lr: 1.0000e-04
